# `git2net` - Extracting and analysing co-editing relationships from *git* repositories

In this tutorial you will learn the basic steps required to obtain a co-editing relationship from a git repoitory using `git2net`.

The tutorial assumes you have `git2net` installed. In addition, it is recommended to create a folder for this tutorial as additional files will be downloaded to your local directory (if not specified otherwise).

```
 +---------------+   git2net          +---------------+
 | Repository on | -----------------> | local         |
 | Github        | clone_repository() | repository    |
 +---------------+   (1)              +---------------+
                                        | git2net 
                                        | mine_git_repo()
                                        v (2)
 +---------------+   pandas           +---------------+
 | pandas        |<------------------ | sqlite data   |
 | dataframe     |  read_sql_query()  | base          |
 +---------------+                    +---------------+ 
      |                                       |
      v                                       v
 Data mining (4)                      Network analysis (3)
```

The following sections are structured according to this process model:

1. Repository Cloning
2. Repository Mining 
3. Network Analysis and visualization (git2net)
4. Data analysis (pandas)

## Repository Cloning

To start, you will need to select and clone a git repository that you are interested in analysing. For the purpose of this tutorial, we will analyse the repository behind `git2net`&mdash;aiming to finally find a solution to the well-known chicken and egg problem.

The following lines will clone the `git2net` repository to your current working directory. To change this location, you can edit the path to the local directory stored in `local_directory`. The folder name of the repository is the name of the repository, which we store in `repo_name`.

In [7]:
import pygit2 as git2
import os
import shutil

git_repo_url = 'https://github.com/gotec/git2net.git'
local_directory = '.'
git_repo_dir = 'git2net4analysis'

if os.path.exists(git_repo_dir):
    shutil.rmtree(git_repo_dir)

repo = git2.clone_repository(git_repo_url, git_repo_dir) # Clones a non-bare repository

Private repositories require some more efforts. Firstly, you have to generate a personal token. The procedure on GitHub side is explained [here](https://docs.github.com/en/github/authenticating-to-github/creating-a-personal-access-token). Make sure to copy your new access token to a file (`secret.txt` for instance). You won’t be able to see it again! Please add `secret.txt` directly to your `.gitignore` file! You wouldn't believe how many access tokens are freely available at github :-)

Now we are able to pass the token as a third parameter embedded in a callback method to `clone_repository()`.

*The code below is commented to allow for execution of the full notebook. Uncomment it to use your own private repository for the tutorial.*

In [ ]:
# import pygit2 as git2
# import os
# import shutil

# git_repo_url = 'https://github.com/user/SecretRepository.git'   # does not exist :-)
# local_directory = '.'
# git_repo_dir = 'secretRepository'

# f = open("secret.txt", "r")
# token = f.read()

# if os.path.exists(git_repo_dir):
#     shutil.rmtree(git_repo_dir)

# callbacks = git2.RemoteCallbacks(git2.UserPass(token, 'x-oauth-basic'))
# repo = git2.clone_repository(git_repo_url, git_repo_dir, callbacks=callbacks)

## Repository Mining

Now that we have obtained a local copy of the repository, we can use `git2net` to obtain a database containing information on all commits and edits made to obtain the current state of the repository.

To do so, we use the `mine_git_repo` function. This function takes two required inputs as well as a number of optional inputs, some of which we will further explore later in this tutorial. Let's start with the required inputs. Here, we need to supply a path to the git repositoy that will be analysed. Below, this is done with the variable `repo_name`. In addition, `git2net` requires a path to the *sqlite* database that will be filled during the mining process. This path is provided as `sqlite_db_file`.

Note, that if no database exists on the supplied path, `git2net` will create a new database. If a database exists, `git2net` will check if the database was mined with the same setting and on the same repository and subsequently resume the mining process from wherever it was left off.

Let's try this out. Below we import `git2net` and point it to the path to which we cloned the database. In addition, we specify the location of the database file in which the results of the mining process will be stored and ensure the database does currently not exist. We then run the `mine_git_repo` function with the optional argument `max_modifications = 1`. With this only commits in which 1 or less files were modified are mined.

In [ ]:
import git2net

sqlite_db_file = 'git2net.db'

# Remove database if exists
if os.path.exists(sqlite_db_file):
    os.remove(sqlite_db_file)

max_modifications = 1
    
git2net.mine_git_repo(git_repo_dir, sqlite_db_file, max_modifications=max_modifications)

While mining, `git2net` provides information about the current progress. The first line shows that no database was found at the current path and mining will be started from scratch. This is totally expected, as we deliberately deleted any existing database before the run.

Subsequently, progress updates on the mining process are printed. The first information denotes the number of processes `git2net` spawns and runs on. `git2net` is highly parallelised and will automatically detect the number of threads of your CPU, fully utilising all of them during operation. In case you want to reduce this load, this can be done by specifically setting the number of processes with the `no_of_processes` option of the `mine_git_repo` function.

The other output shows the number of commits and total number of commits mined in this run, as well as the elapsed time and an estimate of the remaining time to finish.

If a commit is skipped, the reason and the commit hash are printed. Currently, there are three cases in which a commit can be skipped. Firstly, as seen above a commit can exceed the maximum number of modifications set by `max_modifications`. Secondly, processing the commit can take longer as a maximum time defined by the `timeout` option. Thirdly, a commit can be skipped due to an error occuring within the commit. In these cases, please report the repository and commit hash in a new issue on github.com/gotec/git2net.

Let's resume the mining process while increasing the maximum number of modifications to 5!

In [ ]:
max_modifications = 5

git2net.mine_git_repo(git_repo_dir, sqlite_db_file, max_modifications=max_modifications)

As you can see from the output above, the process was resumed from the old database, skipping the already processed commits in the repository.

Great, we made some progress and a large amount of the commits in the repository are already mined and in the database! But what about the other ones? We get some more information on the commits missing from the database from the `mining_state_summary` function. Similar to `mine_git_repo`, it also requires the paths to the repository as well as the database.

In [ ]:
git2net.mining_state_summary(git_repo_dir, sqlite_db_file)

The function again provides a summary of the mining state, as well as details on all missing commits. Let's assume, we are very interest in commit *090c00c342283134a23900f85c1d232499617365* but want to avoid crawling the other missing commits. While this is uneccessary for small repositories such as `git2net` this might become higly relevant for larger projects such as `linux`, where individual commits can make changes to thousands of files which in turn require significant computational resources to analyse. This is particularly important for merge commits, as all files included in the diffs to both parent commits need to be considered. Therefore, for larger projects I generally recommend to run `git2net` with `max_modifications = 1000`, subsequently increasing this number if required.

But now back to mining specifically commit *090c00c342283134a23900f85c1d232499617365*, which can be done with the `commits` option in `mine_git_repo`. We also set the number of processes to 1, enabling serial mode, which can be very helpful for debugging as significantly more information is printed.

In [13]:
# mine_git_repo takes list of commits
commits = ['090c00c342283134a23900f85c1d232499617365']

git2net.mine_git_repo(git_repo_dir, sqlite_db_file, commits=commits, no_of_processes=1)

Found a matching database on provided path. Skipping 218 (95.61%) of 228 commits. 10 commits remaining.


Serial: 100%|██████████| 1/1 [00:00<00:00,  1.54it/s]


Congratulations, you have now mined your first git repository using `git2net`! Note, though that not all commits have been mined at this point. This will be done at a later stage of this tutorial.

## Network Analysis and visualization

You can now use the database to query various information on different commits or edits. In addition, `git2net` also provides the functionality to generate various network projections of the data.

To start, lets try to obtain a co-editing network for our project. This is as simple as calling the `get_coediting_network` function and providing the database we just mined. Before this, we disambiguate author aliases in the database using the tool `gambit` ([source](https://github.com/gotec/gambit)). To do this, we call the function `disambiguate_aliases_db` on our SQLite database that we just mined.

In [ ]:
git2net.disambiguate_aliases_db(sqlite_db_file)
t, node_info, edge_info = git2net.get_coediting_network(sqlite_db_file)
t

The function returns a `pathpy` temporal network object as well as two dictionaries which can be used to return properties of nodes and edges. As of writing this tutorial not all of them are used but they are set as placeholders for future versions of `git2net`.

A `pathpy` temporal network object can be visualised by itself as shown above. In addition, we can also aggregate the network, by dropping the order of events, yielding a standard network object. Let's do this next.

In [ ]:
import pathpy as pp
pp.Network.from_temporal_network(t)

In both the temporal and aggregated network, a node represents an author, whereas edges point from the person changing a line of code to the person who was the original author.

Next, we could ask the question which those files were that authors collaborated on. Therefore, we can plot a bipartite network containing both files and authors as nodes.

In [ ]:
t, node_info, edge_info = git2net.get_bipartite_network(sqlite_db_file)
n = pp.Network.from_temporal_network(t)
n

For this network, `node_info` contains the classes of authors in the network. These can e.g. be used to color nodes as shown below.

In [ ]:
colour_map = {'author': '#73D2DE', 'file': '#2E5EAA'}
node_color = {node: colour_map[node_info['class'][node]] for node in n.nodes}
pp.visualisation.plot(n, node_color=node_color)

The projection of this network that links authors editing the same file is the co-authorship network.

In [ ]:
n, node_info, edge_info = git2net.get_coauthorship_network(sqlite_db_file)
n

Note that it looks similar, however, all information on the direction of interactions is lost.

If we are interested in e.g more recently edited files, we can filter the database by providing the `time_from` and `time_to` options. Let's check the files edited since May 2019.

In [ ]:
from datetime import datetime
time_from = datetime(2019, 5, 1)
t, node_info, edge_info = git2net.get_bipartite_network(sqlite_db_file, time_from=time_from)
n = pp.Network.from_temporal_network(t)
colour_map = {'author': '#73D2DE', 'file': '#2E5EAA'}
node_color = {node: colour_map[node_info['class'][node]] for node in n.nodes}
pp.visualisation.plot(n, node_color=node_color)

`git2net` allows the extraction of editing paths on the level of individual lines. I.e. we are able to track consecutive changes made to a single line over time&mdash;even if these lines move up or down in a file, or even across files. This is very powerful, as it allows us to determine editing sequences as well as find lines that require more editing than others. These could either be very difficult lines to implement or contain very important information, such as the version number in an `__init__.py` file.

To extract these paths, we can use the `get_line_editing_paths` function. As these networks tend be very large we limit the analysis to a very small file for this tutorial. To only look at a specific set of file paths we can use the `file_paths` option.

In [ ]:
file_paths = ['setup.py']
paths, dag, node_info, edge_info = git2net.get_line_editing_paths(sqlite_db_file, git_repo_dir,
                                                                  file_paths=file_paths)
pp.visualisation.plot(dag, node_color=node_info['colors'])

Notice, that despite only looking at a single file the network shown above is not connected. This is due to our database not being complete. Let's fix this now and try again.

In [ ]:
git2net.mine_git_repo(git_repo_dir, sqlite_db_file)

As we now have more commits in our database, we also need to rerun the disambiguation.

In [ ]:
git2net.disambiguate_aliases_db(sqlite_db_file)

Let's color the nodes by type and look at the visualisation again.

In [ ]:
paths, dag, node_info, edge_info = git2net.get_line_editing_paths(sqlite_db_file, git_repo_dir,
                                                                  file_paths=file_paths)
colors = {}
for x in dag.nodes:
    colors[x] = '#FBB13C'
for x in dag.roots:
    colors[x] = '#A83236'
for x in dag.leafs:
    colors[x] = '#21830'

pp.visualisation.plot(dag, node_color=colors, width=1000, height=1000)

As mentioned before, these networks get very large very quickly. Therefore, it is often more useful to work with the `pathpy` path object that is also returned by the function. It cointains all paths and subpaths contained in the network shown above. More information regarding this object can be found in the documentation on [pathpy.net](http://www.pathpy.net/).

This concludes this tutorial, which I hope you found useful. Enjoy using `git2net` and best of luck for your research! If you find any bugs with the code please let me know on [github.com](https://github.com/gotec/git2net).

`git2net` has been developed as open source project. This means your ideas and inputs are highly welcome. Feel free to share the project and contribute yourself. You can imediately get started on the repository you just downloaded!

## Data mining

Let's take a view on data base content. Obviously, we found two tables. The second code snippet illustrates the usage of sql statements to evaluate the content. We extract the unique author names from the `commit` list.

In [ ]:
import sqlite3

db = sqlite3.connect(sqlite_db_file)
cursor = db.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
cursor.fetchall()

In [27]:
cursor.execute("SELECT DISTINCT author_name FROM commits")
rows = cursor.fetchall()

rows

[('Ingo Scholtes',),
 ('Christoph Gote',),
 ('Jan Philip',),
 ('Sebastian Zug',),
 ('Milo Trujillo',),
 ('Breee',)]

Let's take a view on an alternative approach using python pandas. We load the data base in a pandas DataFrame in a first step. Afterwards the evaluation is much more comfortable due to the fact that the pandas API includes a huge amount of tools for statistical analysis and visualization.

In [ ]:
import pandas as pd 

db = sqlite3.connect(sqlite_db_file)
pdCommits = pd.read_sql_query("SELECT * FROM commits", db)

pdCommits.head(5)

The headline lists all extracted features contained in `pdCommits`. How many records/commits are in the history?

In [ ]:
pdCommits.shape

The first number reflects the number of records, the second how man columns/categories are involved.

In [ ]:
pdCommits.author_name.unique()

How many commits did the individual authors contribute to the overall project?

In [ ]:
pdCommits.groupby('author_name')['hash'].count()

That's easy. We want to give a last example that applies the features of pandas. At which time git2net commits are usually submitted? Of course, GitHub and GitLab offers similar visualizations, but based on git2net and pandas you are able to filter your data set individually.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

pdCommits['timestamp'] = pd.to_datetime(pdCommits['author_date'], format="%Y-%m-%d %H:%M:%S")
pdCommits['hours'] = pdCommits.timestamp.dt.hour
pdCommits['days'] = pdCommits.timestamp.dt.dayofweek
fig, ax = plt.subplots(figsize=(18,6))
sns.heatmap(pdCommits.groupby(['days', 'hours'])['timestamp'].count().unstack(), annot=True, ax=ax)

The diagram shows the week with Monday (0) to Sunday (6). Someone seams to work at the weekend too :-) Find out who it is!

## Changes Over Time

It is also possible to examine how collaboration changes over time in a repository.

In [33]:
from collections import defaultdict
WINDOW_SIZE = 365*24*60*60
STEP_SIZE = 30*24*60*60

def generate_plots(t):
    data = defaultdict(list)
    for network, window in pp.RollingTimeWindow(t, window_size=WINDOW_SIZE, step_size=STEP_SIZE,
                                                directed=True, return_window=True):
        data['number of developers'].append(network.ncount())
        data['unique relations directed'].append(network.ecount())
        data['mean outdegree'].append(pp.algorithms.statistics.mean_degree(network, degree='outdegree'))
        data['time'].append(window[1]) # append window end time

    # Plot time-variable network measures
    df = pd.DataFrame(data, columns=list(data.keys()))
    df.set_index(pd.to_datetime(df.time, unit='s'), inplace=True)
    df.drop('time', axis=1, inplace=True)
    df.plot(y='number of developers')
    df.plot(y='unique relations directed')
    df.plot(y='mean outdegree')



The function above generates a dataframe, from which one can plot the number of developers, unique number of relations directed and mean outdegree for the bipartite graph.

In [ ]:
time_from = datetime(2019, 5, 1)
t, node_info, edge_info = git2net.get_bipartite_network(sqlite_db_file, time_from=time_from)
generate_plots(t)